### Logika modelu
- Vyberu si skupinu čtenářů kteří preferují určité knihy, autora, žánr... V tomto případě Tolkiena
- Zjistím jak hodnotili jiné knihy, které nespadají do výše uvedené preference
- Pokud se mi líbí knihy od Tolkiena a jim také, tak ostatní knihy, kterým dali vysoké hodnocení by se pravděpodobně mohli líbit i mě
- Zjistím průměrné hodhocení čtenářů Tolkiena u ostatních knih, které hodnotili 
- Porovnám kde spolu hodnocení nejvíce korelují
- Doporučím další knihy na základě průměrného hodnocení čtenářů Tolkiena
</br></br></br>

### Plus
- doporučuje oblíbené knihy na základě preferencí jedné skupiny hodnotitelů (niche) - Colaborative filtering
</br></br>

### Minus
- doporučuje pouze populární knihy v této skupině, neumožňuje objevovat "jiný žánr" nebo nejpopulárnější knihy obecně
 

### Code review
- nejsou definované datatypy při importu, zbytečně zpomaluje import a hází warning panda je musí hádat - dobré definovat při importu
- původní názvy buněk jsou zbytečně komplikované pro další práci - dobré přejmenovat
- url buňky nepotřebuju, zbytečně mě budou zabírat místo a rušit
- u filtru skupiny čtenářů Tolkiena není definováno jaké dali knize hodnocení. Jak máme vedět jestli se jim líbili nebo ne. Vybral bych ty kteří dali hodnocení > x ( např.:7 a vyšší)
- hodnocení ostatních knih - chaos, zbytečně nepřehledný DF (osobní preference)
- v kódu jsou komentáře
- v pivot jsou Nan, může způsobovat chyby při dalším přípdadném zpracování, replace with 0
- vlastní nápad porovnávání korelací




In [163]:
# import
import pandas as pd
import numpy as np

In [127]:
# load ratings
ratings = pd.read_csv('BX-Book-Ratings.csv', encoding='cp1251', sep=';')

# vyfiltrované hodnocení > 0  -OK
ratings = ratings[ratings['Book-Rating']!=0]
# load books
books = pd.read_csv('BX-Books.csv',  encoding='cp1251', sep=';',error_bad_lines=False)
books.head()

f:\#My Python\Django Projects\My Projects\BookRecommender\env\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [128]:
#users_ratigs = pd.merge(ratings, users, on=['User-ID'])
dataset = pd.merge(ratings, books, on=['ISBN'])
# lower case format
dataset_lowercase=dataset.apply(lambda x: x.str.lower() if(x.dtype == 'object') else x)

In [129]:
dataset_lowercase.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276726,0155061224,5,rites of passage,judith rae,2001,heinle,http://images.amazon.com/images/p/0155061224.0...,http://images.amazon.com/images/p/0155061224.0...,http://images.amazon.com/images/p/0155061224.0...
1,276729,052165615x,3,help!: level 1,philip prowse,1999,cambridge university press,http://images.amazon.com/images/p/052165615x.0...,http://images.amazon.com/images/p/052165615x.0...,http://images.amazon.com/images/p/052165615x.0...
2,276729,0521795028,6,the amsterdam connection : level 4 (cambridge ...,sue leather,2001,cambridge university press,http://images.amazon.com/images/p/0521795028.0...,http://images.amazon.com/images/p/0521795028.0...,http://images.amazon.com/images/p/0521795028.0...
3,276744,038550120x,7,a painted house,john grisham,NaN,doubleday,http://images.amazon.com/images/p/038550120x.0...,http://images.amazon.com/images/p/038550120x.0...,http://images.amazon.com/images/p/038550120x.0...
4,11676,038550120x,10,a painted house,john grisham,NaN,doubleday,http://images.amazon.com/images/p/038550120x.0...,http://images.amazon.com/images/p/038550120x.0...,http://images.amazon.com/images/p/038550120x.0...


In [130]:
# skupina čtenářů tolkiena
tolkien_readers = dataset_lowercase['User-ID'][(dataset_lowercase['Book-Title']=='the fellowship of the ring (the lord of the rings, part 1)') & (dataset_lowercase['Book-Author'].str.contains("tolkien"))]

In [131]:
tolkien_readers

28897     278246
28898       1674
28899      11676
28900     111847
28901     136491
           ...  
309382    116123
309383    116904
309384    184279
351647    189835
370086    237451
Name: User-ID, Length: 205, dtype: int64

In [132]:
tolkien_readers = tolkien_readers.tolist()


In [133]:
len(tolkien_readers)

205

In [134]:
tolkien_readers = np.unique(tolkien_readers)
len(tolkien_readers)

203

In [135]:
# final dataset
books_of_tolkien_readers = dataset_lowercase[(dataset_lowercase['User-ID'].isin(tolkien_readers))]

In [136]:
books_of_tolkien_readers

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
4,11676,038550120x,10,a painted house,john grisham,NaN,doubleday,http://images.amazon.com/images/p/038550120x.0...,http://images.amazon.com/images/p/038550120x.0...,http://images.amazon.com/images/p/038550120x.0...
41,116904,038550120x,10,a painted house,john grisham,NaN,doubleday,http://images.amazon.com/images/p/038550120x.0...,http://images.amazon.com/images/p/038550120x.0...,http://images.amazon.com/images/p/038550120x.0...
115,11676,0671537458,8,waiting to exhale,terry mcmillan,NaN,pocket,http://images.amazon.com/images/p/0671537458.0...,http://images.amazon.com/images/p/0671537458.0...,http://images.amazon.com/images/p/0671537458.0...
133,11676,0679776818,8,birdsong: a novel of love and war,sebastian faulks,NaN,vintage books usa,http://images.amazon.com/images/p/0679776818.0...,http://images.amazon.com/images/p/0679776818.0...,http://images.amazon.com/images/p/0679776818.0...
149,226965,0679776818,5,birdsong: a novel of love and war,sebastian faulks,NaN,vintage books usa,http://images.amazon.com/images/p/0679776818.0...,http://images.amazon.com/images/p/0679776818.0...,http://images.amazon.com/images/p/0679776818.0...
...,...,...,...,...,...,...,...,...,...,...
383657,276050,0679428461,8,model behavior: a novel and 7 stories,jay mcinerney,NaN,random house inc,http://images.amazon.com/images/p/0679428461.0...,http://images.amazon.com/images/p/0679428461.0...,http://images.amazon.com/images/p/0679428461.0...
383658,276050,0679741852,8,the divine sarah: a life of sarah bernhardt,arthur gold,NaN,vintage books usa,http://images.amazon.com/images/p/0679741852.0...,http://images.amazon.com/images/p/0679741852.0...,http://images.amazon.com/images/p/0679741852.0...
383659,276050,0802151094,4,"the woods, lakeboat, edmond",david mamet,NaN,grove press,http://images.amazon.com/images/p/0802151094.0...,http://images.amazon.com/images/p/0802151094.0...,http://images.amazon.com/images/p/0802151094.0...
383708,276313,038039362x,5,the twelfth planet : book i of the earth chron...,zecharia sitchin,NaN,avon,http://images.amazon.com/images/p/038039362x.0...,http://images.amazon.com/images/p/038039362x.0...,http://images.amazon.com/images/p/038039362x.0...


In [137]:
# Number of ratings per other books in dataset
number_of_rating_per_book = books_of_tolkien_readers.groupby(['Book-Title']).agg('count').reset_index()

In [138]:

x = number_of_rating_per_book["User-ID"]
x.sort_values()

0          1
11194      1
11195      1
11196      1
11197      1
        ... 
5770      37
13813     38
14995     73
15616     75
13441    205
Name: User-ID, Length: 17613, dtype: int64

In [164]:
#select only books which have actually higher number of ratings than threshold
books_to_compare = number_of_rating_per_book['Book-Title'][number_of_rating_per_book['User-ID'] >= 8]
books_to_compare

41                                                    1984
44                                     1st to die: a novel
454                                        a painted house
592                                         a time to kill
657                                      a wrinkle in time
                               ...                        
16255                                to kill a mockingbird
16486    tuesdays with morrie: an old man, a young man,...
16781                                     violets are blue
16925                                       watership down
17137    where the heart is (oprah's book club (paperba...
Name: Book-Title, Length: 77, dtype: object

In [140]:
books_to_compare = books_to_compare.tolist()

In [141]:
books_to_compare

['1984',
 '1st to die: a novel',
 'a painted house',
 'a time to kill',
 'a wrinkle in time',
 'angels &amp; demons',
 "bridget jones's diary",
 'dolores claiborne',
 'dreamcatcher',
 "ender's game (ender wiggins saga (paperback))",
 'fahrenheit 451',
 'firestarter',
 'girl with a pearl earring',
 'hannibal',
 'harry potter and the chamber of secrets (book 2)',
 'harry potter and the goblet of fire (book 4)',
 'harry potter and the order of the phoenix (book 5)',
 'harry potter and the prisoner of azkaban (book 3)',
 "harry potter and the sorcerer's stone (book 1)",
 "harry potter and the sorcerer's stone (harry potter (paperback))",
 'hearts in atlantis',
 'house of sand and fog',
 'insomnia',
 'interview with the vampire',
 'it',
 'jurassic park',
 'life of pi',
 'lightning',
 'lord of the flies',
 'pet sematary',
 'red dragon',
 'skipping christmas',
 'sphere',
 'stardust',
 'the bad beginning (a series of unfortunate events, book 1)',
 'the brethren',
 'the client',
 'the da vinci 

In [142]:
ratings_data_raw = books_of_tolkien_readers[['User-ID', 'Book-Rating', 'Book-Title']][books_of_tolkien_readers['Book-Title'].isin(books_to_compare)]

In [143]:
ratings_data_raw.tail(100)

,User-ID,Book-Rating,Book-Title
184466,254206,10,the fellowship of the ring (the lord of the ri...
184467,23571,10,"the two towers (the lord of the rings, part 2)"
184468,170513,10,"the two towers (the lord of the rings, part 2)"
184469,254206,10,"the two towers (the lord of the rings, part 2)"
184837,23699,10,a wrinkle in time
...,...,...,...
351649,189835,5,"the return of the king (the lord of the rings,..."
370086,237451,8,the fellowship of the ring (the lord of the ri...
370087,237451,9,"the return of the king (the lord of the rings,..."
370088,237451,10,"the two towers (the lord of the rings, part 2)"


In [144]:
# group by User and Book and compute mean
ratings_data_raw_nodup = ratings_data_raw.groupby(['User-ID', 'Book-Title'])['Book-Rating'].mean()

In [145]:
ratings_data_raw_nodup.head(50) 

User-ID  Book-Title                                                              
254      1984                                                                         9.000000
         harry potter and the chamber of secrets (book 2)                             9.000000
         harry potter and the goblet of fire (book 4)                                 9.000000
         harry potter and the prisoner of azkaban (book 3)                            9.000000
         harry potter and the sorcerer's stone (book 1)                               9.000000
         stardust                                                                     8.000000
         the fellowship of the ring (the lord of the rings, part 1)                   8.000000
         the golden compass (his dark materials, book 1)                              8.500000
         the hobbit: or there and back again                                          9.000000
         the lion, the witch, and the wardrobe (the chronicles 

In [146]:
# reset index to see User-ID in every row
ratings_data_raw_nodup = ratings_data_raw_nodup.to_frame().reset_index()

In [147]:
ratings_data_raw_nodup

,User-ID,Book-Title,Book-Rating
0,254,1984,9.0
1,254,harry potter and the chamber of secrets (book 2),9.0
2,254,harry potter and the goblet of fire (book 4),9.0
3,254,harry potter and the prisoner of azkaban (book 3),9.0
4,254,harry potter and the sorcerer's stone (book 1),9.0
...,...,...,...
1150,276313,the brethren,8.0
1151,276313,the fellowship of the ring (the lord of the ri...,8.0
1152,276313,the summons,8.0
1153,278246,the fellowship of the ring (the lord of the ri...,8.0


In [161]:
dataset_for_corr = ratings_data_raw_nodup.pivot(index='User-ID', columns='Book-Title', values='Book-Rating')

In [165]:
dataset_for_corr.head()

Book-Title,1984,1st to die: a novel,a painted house,a time to kill,a wrinkle in time,angels &amp; demons,bridget jones's diary,dolores claiborne,dreamcatcher,ender's game (ender wiggins saga (paperback)),...,the testament,the tommyknockers,"the two towers (the lord of the rings, part 2)","the vampire lestat (vampire chronicles, book ii)","the waste lands (the dark tower, book 3)",to kill a mockingbird,"tuesdays with morrie: an old man, a young man, and life's greatest lesson",violets are blue,watership down,where the heart is (oprah's book club (paperback))
User-ID,,,,,,,,,,,,,,,,,,,,,
254,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11676,10.0,9.0,9.0,NaN,9.0,8.0,6.333333,8.0,8.666667,9.0,...,7.0,7.5,10.0,8.0,NaN,9.0,9.0,5.5,5.0,10.0
11944,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
LoR_list = ['the fellowship of the ring (the lord of the rings, part 1)']

result_list = []
worst_list = []

for LoR_book in LoR_list:
    
    #Take out the Lord of the Rings selected book from correlation dataframe
    dataset_of_other_books = dataset_for_corr.copy(deep=False)
    dataset_of_other_books.drop([LoR_book], axis=1, inplace=True)
      
    # empty lists
    book_titles = []
    correlations = []
    avgrating = []

    # corr computation
    for book_title in list(dataset_of_other_books.columns.values):
        book_titles.append(book_title)
        correlations.append(dataset_for_corr[LoR_book].corr(dataset_of_other_books[book_title]))
        tab=(ratings_data_raw[ratings_data_raw['Book-Title']==book_title].groupby(ratings_data_raw['Book-Title']).mean())
        avgrating.append(tab['Book-Rating'].min())
    # final dataframe of all correlation of each book   
    corr_fellowship = pd.DataFrame(list(zip(book_titles, correlations, avgrating)), columns=['book','corr','avg_rating'])
    corr_fellowship.head()

    # top 10 books with highest corr
    result_list.append(corr_fellowship.sort_values('corr', ascending = False).head(10))
    
    #worst 10 books
    worst_list.append(corr_fellowship.sort_values('corr', ascending = False).tail(10))

result_list

[                                                 book      corr  avg_rating
 33                                           stardust  0.909450    7.500000
 38  the drawing of the three (the dark tower, book 2)  0.907758    8.000000
 53                              the phantom tollbooth  0.896262    8.500000
 24                                                 it  0.887229    8.333333
 66                                      the testament  0.852803    8.000000
 4                                   a wrinkle in time  0.848478    8.357143
 9       ender's game (ender wiggins saga (paperback))  0.836660    9.307692
 29                                       pet sematary  0.830264    7.636364
 47                the hobbit: or there and back again  0.820499    8.800000
 40                                           the gift  0.808511    7.750000]